<a href="https://colab.research.google.com/github/srenna/RAGOpenAI/blob/main/RAG_with_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install langchain --progress-bar off
!pip install openai --progress-bar off
!pip install tiktoken --progress-bar off
!pip install faiss-gpu --progress-bar off
!pip install langchain_experimental --progress-bar off
!pip install "langchain[docarray]" --progress-bar off
!pip install -qqq pypdf==3.15.0 --progress-bar off
!pip install -U langchain-openai --progress-bar off

In [38]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken
import os
import re

In [ ]:
# Prompt the user for their OpenAI API key
api_key = input("Please enter your OpenAI API key: ")

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = api_key

# Optionally, check that the environment variable was set correctly
print("OPENAI_API_KEY has been set!")


In [19]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain_openai import OpenAIEmbeddings

In [20]:
# load the directory
pdf_file_path = '/content/drive/MyDrive/POVs/Test PDF'
loader = PyPDFDirectoryLoader(pdf_file_path)
data = loader.load()

print(len(data))
# chunk the data
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
data = text_splitter.split_documents(data)

16


In [21]:
# Define a function to clean the text
def clean_text(text):
    # Remove '\xa0' (non-breaking space)
    text = re.sub(r'\xa0', ' ', text)

    # Remove '\n' (newlines)
    text = re.sub(r'\n', ' ', text)

    # Remove consecutive dots ('.....') with just one dot
    text = re.sub(r'\.{2,}', '', text)

    # Remove '\uf0b7'
    text = re.sub(r'\uf0b7', ' ', text)

    # Remove consecutive spaces
    text = re.sub(r' +', ' ', text)

    return text

# Clean the text in the 'page_content' of each document in the 'data' list
import re

for document in data:
    document.page_content = clean_text(document.page_content)

In [ ]:
# preview some data
# data

In [23]:
# Create vector store
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embedding=embeddings)

In [24]:
# Create conversation chain
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4")
memory = ConversationBufferMemory(
memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        memory=memory
        )

In [25]:
# query = "When does the new Standard is effective for reporting periods begin"
# result = conversation_chain({"question": query})
# answer = result["answer"]
# answer

In [26]:
# query = "How much is the financial services industry's spending on quantum computing expected to grow by"
# result = conversation_chain({"question": query})
# answer = result["answer"]
# answer

In [27]:
# query = "US electricity sales continued to rise as the pandemic recovery progressed. By how much did they increase?"
# result = conversation_chain({"question": query})
# answer = result["answer"]
# answer

In [28]:
query = "What are the upstream cyberthreats?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

'The upstream cyber threats mentioned are misappropriation of commercially sensitive reservoir performance and well data, which can damage the operator’s competitive position in a play or field, and key well-control and safety equipment being pushed outside normal operating parameters or shut down completely, which can lead to operational shutdown and financial loss, as well as field safety incidents, due to equipment malfunction.'

In [41]:
query = "Who are the authors of this document? There are two authors."
result = conversation_chain({"question": query})
answer = result["answer"]
answer

'The document does not provide the names of the authors specifically discussing upstream cyberthreats. However, it mentions Paul Zonneveld as the Global Energy & Resources Risk Advisory Leader at Deloitte Canada and the source of some information is attributed to Talbot, J, and Jakeman, M.'

In [34]:
query = "Who are the threat actors in the oil and gas industry?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

'The text does not provide specific information on who the individuals or groups are that are responsible for the cyber threats in the oil and gas industry.'

In [31]:
query = "Explain the integrated approach to combat cyber risk proposed by Deloitte."
result = conversation_chain({"question": query})
answer = result["answer"]
answer

"The document doesn't provide a detailed explanation of Deloitte's integrated approach to combat cyber risk. However, it does mention a cyber security maturity model, which includes aspects like awareness and education, network segmentation, annual risk assessment, physical security, vulnerability testing, security monitoring, incident response plans, and protection measures like virus and malware protection, end point security, mobile protection, third party security, and behavioral analytics. It also emphasizes a harmonized and aligned approach that integrates both engineering and IT disciplines. Additionally, it highlights the importance of understanding both the engineering and IT disciplines as well as leading sector-specific cyber security practices."